# AstroHack - Tensorflow + Keras Example

This tutorial will demonstrate how to predict galaxy M/L ratios using a simple convolutional neural network with the Keras API of TensorFlow 2.0.

1. [Setup](#1.-Setup)    
    1.1. [Download data](#1.1.-Download-data)    
    1.2. [Install dependencies](#1.2.-Install-dependencies)
    
2. [Prepare data](#2.-Prepare-data)    
    2.1. [Load labels and metadata](#2.1.-Load-labels-and-metadata)    
    2.2. [Load images into np.arrays](#2.2.-Load-images-into-numpy-arrays)    
    2.3. [Split validation set from training set](#2.3.-Split-validation-set-from-training-set)    
3. [Build a simple convolutional neural network](#3.-Build-a-simple-convolutional-neural-network)
4. [Generate predictions](#4.-Generate-predictions)


## 1. Setup

### 1.1. Download data

The AstroHack data files do not come along with this repository, you need to download them into the `./data` directory before running this notebook.

#### If you prefer to download them manually:

Please follow the instructions of `./data/README.md` to setup the data folder.


#### If you're running this notebook from a AWS SageMaker notebook:

1. Identify the S3 bucket name from the links to data files.     
   For example, the bolded part in the URI "__astrohack-public-data__.s3.amazonaws.com/train_metadata.tsv" is the bucket name.
   
2. Replace the `<bucket_name>` placeholder in the next cell with the actual bucket name.
3. Run the next cell to download and unpack data files.

In [ ]:
# If running from a SageMaker notebook instance
# Update the <bucket_name> placeholder below before execution!
!aws s3 cp s3://<bucket_name>/ ./data --recursive
    
!unzip -q ./data/eval_images_69pix.zip -d ./data
!unzip -q ./data/train_images_69pix.zip -d ./data

### 1.2. Install dependencies

Run the following command to install Python dependencies.

In [ ]:
!pip install -r requirements.txt -q

## 2. Prepare data

We need to load and parse the data files before we can run any machine learning model.

### 2.1. Load labels and metadata

Pandas is handy for parsing tabular data and we will use it to load the metadata files.

In [ ]:
import pandas as pd
import os

# Read metadata TSV files
train_labels = pd.read_csv('./data/train_metadata.tsv', sep='\t')
eval_labels = pd.read_csv('./data/eval_metadata.tsv', sep='\t')

# Print the shape
print('Training metadata shape: {}'.format(train_labels.shape))
print('Evaluation metadata shape: {}'.format(eval_labels.shape))

# Show head of training metadata
train_labels.head(3)

Shown above are the top 3 rows in the training metadata files with columns:

- `SDSS_ID`: the unique identifier for each galaxy used by SDSS database.
- `M/L`: the ground truth Mass-to-Luminosity ratio.
- `L_g`: luminosity of the galaxy in g-band wavelength. *Not used in this tutorial.*
- `distance_Mpc`: the distance between the subject galaxy and Earth in megaparsec. *Not used in this tutorial.*
- `galsize_kpc`: the size of the subject galaxy in kiloparsec. *Not used in this tutorial.*
- `image_name`: the filename of the corresponding image file.

In this tutorial, we will use the image as the only feature to predict the M/L ratio.

### 2.2. Load images into numpy arrays

In this tutorial, we will only use the 69x69pix galatic images.

In [ ]:
# Locate training and evaluation images
train_img_folder = './data/train_images_69pix'
eval_img_folder = './data/evaluation_images_69pix'

print('Found {} training images'.format(len(os.listdir(train_img_folder))))
print('Found {} evaluation images'.format(len(os.listdir(eval_img_folder))))

In [ ]:
import numpy as np
from PIL import Image

# Load train and eval images into numpy arrays
train_image_paths = [train_img_folder + '/' + img_name for img_name in train_labels['image_name']]
eval_image_paths = [eval_img_folder + '/' + img_name for img_name in eval_labels['image_name']]

train_images = np.array([np.array(Image.open(img_path)) for img_path in train_image_paths])
eval_images = np.array([np.array(Image.open(img_path)) for img_path in eval_image_paths])

# Expand the channel dimension
train_images = np.expand_dims(train_images, axis=3)
eval_images = np.expand_dims(eval_images, axis=3)

# Show shape
print('Training image array shape: {}'.format(train_images.shape))
print('Evaluation image array shape: {}'.format(eval_images.shape))

In [ ]:
import matplotlib.pyplot as plt

# Preview some of the images
_show_n_images = 4
fig, axs = plt.subplots(ncols=_show_n_images, figsize=(3 * _show_n_images, 3))
for i in range(_show_n_images):
    axs[i].imshow(train_images[i, :, :, 0], cmap='gray')
    axs[i].set_title('{}\nM/L = {:.4f}'.format(train_labels['SDSS_ID'].iloc[i], 
                                               train_labels['M/L'].iloc[i]))

Each image is 69x69 pix large and there is only one channel (black-and-white), therefore each image can be represented by a tensor with size of (69, 69, 1).

### 2.3. Split validation set from training set

Theoretically we can use the entire training dataset to train our machine learning model, but practically we need to reserve a small portion of it to validate our model and check the model performance. This reserved portion is often referred as the *validation set*. 

In this tutorial, we will use 10% of the training data as the validation set, and use the remaining 90% to actually train the model.

In [ ]:
validation_split = 0.1  # Use 10% of training data for validation

In [ ]:
np.random.seed(42)  # seed randomness

# Move 10% of data into validation set
val_index = train_labels.sample(frac=validation_split).index
val_labels = train_labels.loc[val_index, :]
train_labels = train_labels.loc[~train_labels.index.isin(val_index), :]

# Print the shape
print('Training metadata shape: {}'.format(train_labels.shape))
print('Validation metadata shape: {}'.format(val_labels.shape))

Now we have splitted the training and validation sets, we simply need to lookup the corresponding images and M/L values as well.

In [ ]:
X_train = train_images[train_labels.index.values, :, :]
X_val = train_images[val_index.values, :, :]
Y_train = train_labels['M/L'].values
Y_val = val_labels['M/L'].values

## 3. Build a simple convolutional neural network

[Convolutional Neural Network (CNN)](https://en.wikipedia.org/wiki/Convolutional_neural_network) is a family of deep learning models that work well on image data. 

In the below code segments, we will build a simple CNN sturtcure, train it and use it to predict the M/L ratio from galaxy images.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Build a simple sequential CNN model
model = tf.keras.Sequential()

# Add a batch normalization layer
model.add(layers.BatchNormalization(input_shape=(69, 69, 1)))

# CNN layer 1
model.add(layers.Conv2D(32, kernel_size=5, strides=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))  # Dropout is a common technique to prevent overfitting

# CNN layer 2
model.add(layers.Conv2D(64, kernel_size=3, strides=(1, 1), activation='relu'))
model.add(layers.GlobalMaxPooling2D())
model.add(layers.Dropout(0.2))

# Dense layers to collect the convolutional features and make a single output
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='relu'))

# Use MSE as the loss function
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
# Train model
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val),
                    epochs=25, batch_size=64)

In [ ]:
# Visualize the training curve
history_df = pd.DataFrame(history.history)
plt.plot(history_df['loss'], label='training_loss')
plt.plot(history_df['val_loss'], label='validation_loss')
plt.xlabel('epoch')
plt.ylabel('MSE')
_ = plt.legend()

As shown in the training plot, the loss is gradually reduced with more iterations, which indicates the model is learning and making progress.

## 4. Generate predictions

Now our CNN model is trained, we can use it to predict on images in the evaluation set.

In [ ]:
# Generate predictions on the evaluation set
predicted_ml = model.predict(eval_images)

# Compile predictions into the submission format
eval_labels.loc[:, 'pred_ml'] = predicted_ml
eval_submission = eval_labels.loc[:, ['SDSS_ID', 'pred_ml']]
eval_submission.head(3)

In [ ]:
# Save the submission file
eval_submission.to_csv('./eval_submission.tsv', sep='\t', index=False)

Now you can submit this file for evaluation. 